# Data models and context broker
This notebook shows some examples of how to interact with a context broker using the Toolbox components and how to use the defined data models.

## Data models
The Toolbox provides a set of [data models](https://github.com/CommuniCityProject/communicity_toolbox/tree/master/docs/DataModels) that define the structure of the data generated and consumed by the Toolbox components. For each one, a Python class is provided to facilitate its management.

In [1]:
from toolbox import Structures
from toolbox.Context import entity_parser
from toolbox.DataModels import (
    Face, Image, InstanceSegmentation, PersonKeyPoints, DataModelsCatalog)

We can create a new entity as follows:

In [2]:
face = Face(
    image="urn:ngsi-ld:Image:1234",
    bounding_box=Structures.BoundingBox(0.1,0.2,0.3,0.4),
    detection_confidence=0.99,
    age=30,
    emotion=Structures.Emotion.HAPPINESS
)

print(face.pretty())

--------------------------------------Face--------------------------------------
id				None
dateObserved			2023-06-16 12:38:41.216313
type				Face
image				urn:ngsi-ld:Image:1234
bounding_box			BoundingBox(0.100,0.200,0.300,0.400)
detection_confidence		0.99
age				30.0
gender				None
gender_confidence		None
emotion				HAPPINESS
emotion_confidence		None
features			None
features_algorithm		None
recognition_domain		None
recognized			False
recognized_distance		None
recognized_person		None
--------------------------------------------------------------------------------



We can get and modify its attributes.

In [3]:
print(face.age)
print(face.bounding_box)
face.age = 100
print(face.age)

30.0
BoundingBox(0.100,0.200,0.300,0.400)
100


Each data model is registered in the _DataModelsCatalog_. We can use this module to get the appropriate class from each data model name.

In [4]:
print(type(DataModelsCatalog.data_models_catalog))
for k, v in DataModelsCatalog.data_models_catalog.items():
    print(k, v)

<class 'dict'>
InstanceSegmentation <class 'toolbox.DataModels.InstanceSegmentation.InstanceSegmentation'>
PersonKeyPoints <class 'toolbox.DataModels.PersonKeyPoints.PersonKeyPoints'>
Image <class 'toolbox.DataModels.Image.Image'>
Face <class 'toolbox.DataModels.Face.Face'>


We can convert the entities from and to dicts using the ``entity_parser`` module.

In [5]:
json_face = entity_parser.data_model_to_json(face)
print(json_face)

new_face = entity_parser.json_to_data_model(json_face)
print(new_face.pretty())

{'id': 'urn:ngsi-ld:Face:9f0mLgwxEe6LRzCcI__G6g', 'type': 'Face', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateModified': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'image': {'type': 'Relationship', 'object': 'urn:ngsi-ld:Image:1234'}, 'boundingBox': {'type': 'Property', 'value': {'xmin': 0.1, 'ymin': 0.2, 'xmax': 0.3, 'ymax': 0.4}}, 'detectionConfidence': {'type': 'Property', 'value': 0.99}, 'age': {'type': 'Property', 'value': 100}, 'emotion': {'type': 'Property', 'value': 'HAPPINESS'}, 'recognized': {'type': 'Property', 'value': False}}
--------------------------------------Face--------------------------------------
id				urn:ngsi-ld:Face:9f0mLgwxEe6LRzCcI__G6g
dateObserved			2023-06-16 12:38:41
typ

## Context broker
The context broker is used to store and serve the entities produced and consumed by the Toolbox components. The [_ContextCli_](../../toolbox/Context/ContextCli.py) module provides an interface for interacting with it.

In [6]:
from toolbox.Context import ContextCli, Subscription

Create a _ContextCli_ object that connects to a context broker on local host, port 1026

In [7]:
cli = ContextCli(
    "127.0.0.1",
    1026
)

### Post and get a data model
We can post a Toolbox data model to the context broker with the ``post_data_model`` method. Its ID must be unique in the context broker. We can set it to ``None`` to generate a random one.

In [8]:
face.id = None

uploaded_json = cli.post_data_model(face)

face_id = face.id
print(face_id)
print(uploaded_json)

urn:ngsi-ld:Face:9iTERgwxEe6jwjCcI__G6g
{'id': 'urn:ngsi-ld:Face:9iTERgwxEe6jwjCcI__G6g', 'type': 'Face', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateModified': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'image': {'type': 'Relationship', 'object': 'urn:ngsi-ld:Image:1234'}, 'boundingBox': {'type': 'Property', 'value': {'xmin': 0.1, 'ymin': 0.2, 'xmax': 0.3, 'ymax': 0.4}}, 'detectionConfidence': {'type': 'Property', 'value': 0.99}, 'age': {'type': 'Property', 'value': 100}, 'emotion': {'type': 'Property', 'value': 'HAPPINESS'}, 'recognized': {'type': 'Property', 'value': False}}


We can retrieve an entity from the context broker by its ID, using the ``get_entity`` method.

In [9]:
new_face = cli.get_entity(face_id)
print(new_face.pretty())

--------------------------------------Face--------------------------------------
id				urn:ngsi-ld:Face:9iTERgwxEe6jwjCcI__G6g
dateObserved			2023-06-16 12:38:41
type				Face
image				urn:ngsi-ld:Image:1234
bounding_box			BoundingBox(0.100,0.200,0.300,0.400)
detection_confidence		0.99
age				100.0
gender				None
gender_confidence		None
emotion				HAPPINESS
emotion_confidence		None
features			None
features_algorithm		None
recognition_domain		None
recognized			False
recognized_distance		None
recognized_person		None
--------------------------------------------------------------------------------



If we want to retrieve an entity whose data model is not defined in the Toolbox, we can get it as a simple dict:

In [10]:
face_dict = cli.get_entity(face_id, as_dict=True)
print(face_dict)

{'@context': 'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld', 'id': 'urn:ngsi-ld:Face:9iTERgwxEe6jwjCcI__G6g', 'type': 'Face', 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateModified': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-06-16T12:38:41Z'}}, 'image': {'type': 'Relationship', 'object': 'urn:ngsi-ld:Image:1234'}, 'boundingBox': {'type': 'Property', 'value': {'xmin': 0.1, 'ymin': 0.2, 'xmax': 0.3, 'ymax': 0.4}}, 'detectionConfidence': {'type': 'Property', 'value': 0.99}, 'age': {'type': 'Property', 'value': 100}, 'emotion': {'type': 'Property', 'value': 'HAPPINESS'}, 'recognized': {'type': 'Property', 'value': False}}


We can also update an existing entity.

In [11]:
face.age = 30
cli.update_data_model(face)

new_face = cli.get_entity(face_id)
print(new_face.pretty())

--------------------------------------Face--------------------------------------
id				urn:ngsi-ld:Face:9iTERgwxEe6jwjCcI__G6g
dateObserved			2023-06-16 12:38:41
type				Face
image				urn:ngsi-ld:Image:1234
bounding_box			BoundingBox(0.100,0.200,0.300,0.400)
detection_confidence		0.99
age				30.0
gender				None
gender_confidence		None
emotion				HAPPINESS
emotion_confidence		None
features			None
features_algorithm		None
recognition_domain		None
recognized			False
recognized_distance		None
recognized_person		None
--------------------------------------------------------------------------------



Entities can also be deleted by their ID:

In [12]:
cli.delete_entity(face_id)
new_face = cli.get_entity(face_id)
print(new_face)

None


Finally we can get a list of the current entities in the context broker with the methods: ``get_all_entities``, ``iterate_entities``, ``get_entities_page`` and ``get_types``

In [13]:
# Post some entities
for _ in range(5):
    face.id = None
    cli.post_data_model(face)

# Get all the entities
all = cli.get_all_entities(entity_type=face.type)
print("All entities len:", len(all))

# Iterate over all the entities pages
for e in cli.iterate_entities(entity_type=face.type, limit=3):
    print("Iteration page len", len(e))
    break

# Get a single page
page = cli.get_entities_page(entity_type=face.type, limit=2, offset=2)
print("Page len", len(page))

# Get the current entity types on the context broker
types = cli.get_types()
print("Entity types:", types)

All entities len: 5
Iteration page len 3
Page len 2
Entity types: ['Face']


### Subscriptions
The context broker allows us to subscribe to the creation or modification of certain entities and attributes and receive notifications when this happens. We can create a subscription with the method ``subscribe``

In [14]:
sub = Subscription(
    notification_uri="127.0.0.1:9999",
    entity_type="Face"
)
subscription_id = cli.subscribe(sub)
print(subscription_id)

urn:ngsi-ld:Subscription:f6849f58-0c31-11ee-ada5-0242ac140002


It is possible to create duplicate subscriptions that will send the same data more than once. The method ``get_conflicting_subscriptions`` can be used to check for possible conflicting subscriptions before creating them.

In [15]:
conflicts = cli.get_conflicting_subscriptions(sub)
print(conflicts)
print(conflicts[0])

{
    "type": "Subscription",
    "notification": {
        "format": "normalized",
        "endpoint": {
            "uri": "127.0.0.1:9999",
            "accept": "application/json"
        }
    },
    "entities": [
        {
            "type": "Face"
        }
    ],
    "id": "urn:ngsi-ld:Subscription:f6849f58-0c31-11ee-ada5-0242ac140002"
}


As with entities, it is possible to get subscriptions in the context broker with the methods: ``get_subscription``, ``get_subscriptions_page``, ``iterate_subscriptions`` and ``get_all_subscriptions``.

In [16]:
# Create some subscriptions
for _ in range(5):
    cli.subscribe(
        notification_uri="127.0.0.1:9999",
        entity_type="Face"
    )

# Get a single subscription
print(cli.get_subscription(subscription_id))

# Get a single page
page = cli.get_subscriptions_page(limit=2, offset=2)
print("Page len", len(page))

# Iterate over all the subscriptions pages
for e in cli.iterate_subscriptions(limit=3):
    print("Iteration page len", len(e))
    break

# Get all the subscriptions
all = cli.get_all_subscriptions()
print("All subscriptions len:", len(all))

{
    "type": "Subscription",
    "notification": {
        "format": "normalized",
        "endpoint": {
            "uri": "127.0.0.1:9999",
            "accept": "application/json"
        }
    },
    "entities": [
        {
            "type": "Face"
        }
    ],
    "id": "urn:ngsi-ld:Subscription:f6849f58-0c31-11ee-ada5-0242ac140002"
}
Page len 2
Iteration page len 3
All subscriptions len: 6


The IDs of the created subscriptions are stored in the property ``subscription_ids``. We can remove a subscription with the method ``unsubscribe`` or remove all the created subscriptions within the ``ContextCli`` object with ``unsubscribe_all``

In [17]:
print(len(cli.subscription_ids), cli.subscription_ids)

cli.unsubscribe(subscription_id)
print(len(cli.subscription_ids), cli.subscription_ids)

cli.unsubscribe_all()
print(len(cli.subscription_ids), cli.subscription_ids)

6 ['urn:ngsi-ld:Subscription:f6849f58-0c31-11ee-ada5-0242ac140002', 'urn:ngsi-ld:Subscription:f6960018-0c31-11ee-be79-0242ac140002', 'urn:ngsi-ld:Subscription:f696e550-0c31-11ee-a590-0242ac140002', 'urn:ngsi-ld:Subscription:f697d5d2-0c31-11ee-adee-0242ac140002', 'urn:ngsi-ld:Subscription:f698b164-0c31-11ee-b084-0242ac140002', 'urn:ngsi-ld:Subscription:f699a632-0c31-11ee-b0f8-0242ac140002']
5 ['urn:ngsi-ld:Subscription:f6960018-0c31-11ee-be79-0242ac140002', 'urn:ngsi-ld:Subscription:f696e550-0c31-11ee-a590-0242ac140002', 'urn:ngsi-ld:Subscription:f697d5d2-0c31-11ee-adee-0242ac140002', 'urn:ngsi-ld:Subscription:f698b164-0c31-11ee-b084-0242ac140002', 'urn:ngsi-ld:Subscription:f699a632-0c31-11ee-b0f8-0242ac140002']
0 []
